In [1]:
# importing all essential libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Importing Models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb


from sklearn.metrics import mean_squared_error, r2_score 

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import mlflow
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('flight.csv')
df.head(10)

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
5,5,Vistara,UK-945,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.33,1,5955
6,6,Vistara,UK-927,Delhi,Morning,zero,Morning,Mumbai,Economy,2.08,1,6060
7,7,Vistara,UK-951,Delhi,Afternoon,zero,Evening,Mumbai,Economy,2.17,1,6060
8,8,GO_FIRST,G8-334,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.17,1,5954
9,9,GO_FIRST,G8-336,Delhi,Afternoon,zero,Evening,Mumbai,Economy,2.25,1,5954


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        300153 non-null  int64  
 1   airline           300153 non-null  object 
 2   flight            300153 non-null  object 
 3   source_city       300153 non-null  object 
 4   departure_time    300153 non-null  object 
 5   stops             300153 non-null  object 
 6   arrival_time      300153 non-null  object 
 7   destination_city  300153 non-null  object 
 8   class             300153 non-null  object 
 9   duration          300153 non-null  float64
 10  days_left         300153 non-null  int64  
 11  price             300153 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 27.5+ MB


In [5]:
df.shape

(300153, 12)

In [6]:
df.isna().sum()

Unnamed: 0          0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [7]:
df.describe()

,Unnamed: 0,duration,days_left,price
count,300153.000000,300153.000000,300153.000000,300153.000000
mean,150076.000000,12.221021,26.004751,20889.660523
std,86646.852011,7.191997,13.561004,22697.767366
min,0.000000,0.830000,1.000000,1105.000000
25%,75038.000000,6.830000,15.000000,4783.000000
50%,150076.000000,11.250000,26.000000,7425.000000
75%,225114.000000,16.170000,38.000000,42521.000000
max,300152.000000,49.830000,49.000000,123071.000000


In [8]:
#Drop Unnamed value:
df.drop(columns = ["Unnamed: 0"],inplace = True)

In [9]:
df.head(10)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
5,Vistara,UK-945,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.33,1,5955
6,Vistara,UK-927,Delhi,Morning,zero,Morning,Mumbai,Economy,2.08,1,6060
7,Vistara,UK-951,Delhi,Afternoon,zero,Evening,Mumbai,Economy,2.17,1,6060
8,GO_FIRST,G8-334,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.17,1,5954
9,GO_FIRST,G8-336,Delhi,Afternoon,zero,Evening,Mumbai,Economy,2.25,1,5954


In [10]:
df.airline.unique()

array(['SpiceJet', 'AirAsia', 'Vistara', 'GO_FIRST', 'Indigo',
       'Air_India'], dtype=object)

In [11]:
'''
#Define categorical and numerical column names
cat_columns = ['airline','flight','source_city','departure_time','stops','arrival_time','destination_city','class',]
num_columns = ['duration','days_left']
# Select categorical and numerical columns from the DataFrame
cat_df = df[cat_columns]
num_df = df[num_columns]

# Define the StandardScaler and LabelEncoder objects
scaler = StandardScaler()
encoder = LabelEncoder()

# Apply StandardScaler to numerical columns
#num_df_scaled = pd.DataFrame(scaler.fit_transform(num_df), columns=num_df.columns)

# Apply LabelEncoder to categorical columns
cat_df_encoded = cat_df.copy()
for col in cat_columns:
    cat_df_encoded[col] = encoder.fit_transform(cat_df[col])

# Concatenate the scaled numerical and encoded categorical DataFrames
final_df = pd.concat([cat_df_encoded, num_df], axis=1)
'''

In [23]:
#Define categorical and numerical column names
cat_columns = ['airline','flight','source_city','departure_time','stops','arrival_time','destination_city','class',]
num_columns = ['duration','days_left']
# Select categorical and numerical columns from the DataFrame
cat_df = df[cat_columns]
num_df = df[num_columns]

encoder = OrdinalEncoder().fit_transform(df[cat_columns])
encoder = pd.DataFrame(encoder,columns = cat_columns)

X = pd.concat([encoder,df[num_columns]],axis=1)
Y= df['price']

In [12]:
#final_df = pd.DataFrame(scaler.fit_transform(preprocessed_df), columns=preprocessed_df.columns)
#final_df

,airline,source_city,stops,destination_city,class,duration,days_left
0,4,2,2,5,1,2.17,1
1,4,2,2,5,1,2.33,1
2,0,2,2,5,1,2.17,1
3,5,2,2,5,1,2.25,1
4,5,2,2,5,1,2.33,1
...,...,...,...,...,...,...,...
300148,5,1,0,3,0,10.08,49
300149,5,1,0,3,0,10.42,49
300150,5,1,0,3,0,13.83,49
300151,5,1,0,3,0,10.00,49


In [13]:
#X = final_df
#y = df['price']

In [24]:
X

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,4.0,1408.0,2.0,2.0,2.0,5.0,5.0,1.0,2.17,1
1,4.0,1387.0,2.0,1.0,2.0,4.0,5.0,1.0,2.33,1
2,0.0,1213.0,2.0,1.0,2.0,1.0,5.0,1.0,2.17,1
3,5.0,1559.0,2.0,4.0,2.0,0.0,5.0,1.0,2.25,1
4,5.0,1549.0,2.0,4.0,2.0,4.0,5.0,1.0,2.33,1
...,...,...,...,...,...,...,...,...,...,...
300148,5.0,1477.0,1.0,4.0,0.0,2.0,3.0,0.0,10.08,49
300149,5.0,1481.0,1.0,0.0,0.0,5.0,3.0,0.0,10.42,49
300150,5.0,1486.0,1.0,1.0,0.0,5.0,3.0,0.0,13.83,49
300151,5.0,1483.0,1.0,1.0,0.0,2.0,3.0,0.0,10.00,49


In [25]:
y

0          5953
1          5953
2          5956
3          5955
4          5955
          ...  
300148    69265
300149    77105
300150    79099
300151    81585
300152    81585
Name: price, Length: 300153, dtype: int64

In [26]:
unique_values = df['price'].value_counts()
unique_values

price
54608    1445
2339     1442
54684    1390
60978    1383
60508    1230
         ... 
12014       1
26912       1
20145       1
25462       1
7533        1
Name: count, Length: 12157, dtype: int64

In [27]:
# Split the dataset into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [28]:
# Create an experiment (or select an existing one)
mlflow.set_experiment("Flight_Price_Prediction")

# Define regression algorithms and their parameter grids
algorithms = {
    #'Linear Regression': (LinearRegression(), {}),
    #'Ridge Regression': (Ridge(), {'alpha': np.logspace(-3, 3, 7)}),
    #'Lasso Regression': (Lasso(), {'alpha': np.logspace(-3, 3, 7)}),
    #'Decision Tree Regressor': (DecisionTreeRegressor(), {'max_depth': [None, 10, 20, 30, 40], 'criterion': ['mse', 'friedman_mse', 'mae', 'poisson']}),
    'Random Forest Regressor': (RandomForestRegressor(), {'n_estimators': [30],'max_depth': [None],'min_samples_split': [5],'min_samples_leaf': [1],'bootstrap': [True]}),
    #'K-Nearest Neighbors': (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7],'weights': ['uniform', 'distance'],'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],'p': [1, 2]}),
    #'CatBoost Regressor': (CatBoostRegressor(),{'n_estimators': [50, 70, 120],'max_depth': [4, 6, 8],'learning_rate': [0.01, 0.05, 0.1]}),
    #'Gradient Boosting Regressor': (GradientBoostingRegressor(), {'n_estimators': [20, 50, 70],'learning_rate': [0.1, 0.2],'max_depth': [3, 4, 5]}),
    #'XGBoost Regressor': (xgb.XGBRegressor(), {'n_estimators': [30, 60, 90],'learning_rate': [0.01, 0.1, 0.2],'max_depth': [3, 4, 5]}),
}

In [29]:
# Perform RandomizedSearchCV for each algorithm individually
for algorithm_name, (model, param_dist) in algorithms.items():
    with mlflow.start_run():
        random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        random_search.fit(X_train, y_train)
 
        
        # Set tags for an experiment or run
        mlflow.set_tag("source", "notebook")
        mlflow.set_tag("Company", "VS IT Services")
        mlflow.set_tag("Created_By", "Team Aviation")

        
        
        y_pred = random_search.best_estimator_.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Log MSE and R2 metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        
        # Calculate "accuracy" based on a threshold (e.g., predictions within 1 unit of the true values)
        threshold = 5.0
        accuracy = np.mean(np.abs(y_test - y_pred) <= threshold)
        mlflow.log_metric("accuracy", accuracy)  # Log the accuracy

        # Log the threshold as a parameter
        mlflow.log_param("threshold", threshold)
        
        # Log parameters and metrics to MLflow
        mlflow.log_params(random_search.best_params_)
        
        # Log the algorithm and its hyperparameters
        algorithm_info = {
            "algorithm_name": algorithm_name,
            "best_params": random_search.best_params_,
        }
        
        mlflow.log_params(algorithm_info)
        
       # Save the best model
        mlflow.sklearn.log_model(random_search.best_estimator_, algorithm_name)
        
        # Print the results
        print(f"Model: {algorithm_name}")
        print(f"Best Hyperparameters: {random_search.best_params_}")
        print(f"Best MSE: {mse}")
        print(f"Best R2: {r2}")
        print("\n")


Model: Random Forest Regressor
Best Hyperparameters: {'n_estimators': 30, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': True}
Best MSE: 5163219.6125634
Best R2: 0.9899488364922141




In [30]:
X_train

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
187826,5.0,1492.0,1.0,5.0,0.0,5.0,5.0,1.0,23.17,4
64372,1.0,797.0,5.0,1.0,0.0,0.0,4.0,1.0,7.42,18
171630,3.0,387.0,3.0,4.0,0.0,0.0,4.0,1.0,6.58,11
233266,1.0,870.0,5.0,1.0,0.0,2.0,0.0,0.0,9.25,39
2982,2.0,1084.0,2.0,2.0,0.0,3.0,5.0,1.0,6.08,17
...,...,...,...,...,...,...,...,...,...,...
178569,1.0,771.0,3.0,1.0,0.0,5.0,1.0,1.0,17.08,28
31962,1.0,729.0,2.0,2.0,0.0,5.0,3.0,1.0,27.08,26
220060,5.0,1552.0,2.0,2.0,0.0,0.0,3.0,0.0,17.25,18
285919,5.0,1522.0,3.0,0.0,0.0,5.0,1.0,0.0,10.17,15


In [31]:
all_df = pd.concat([X, y], axis=1)

In [32]:
all_df

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,4.0,1408.0,2.0,2.0,2.0,5.0,5.0,1.0,2.17,1,5953
1,4.0,1387.0,2.0,1.0,2.0,4.0,5.0,1.0,2.33,1,5953
2,0.0,1213.0,2.0,1.0,2.0,1.0,5.0,1.0,2.17,1,5956
3,5.0,1559.0,2.0,4.0,2.0,0.0,5.0,1.0,2.25,1,5955
4,5.0,1549.0,2.0,4.0,2.0,4.0,5.0,1.0,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...
300148,5.0,1477.0,1.0,4.0,0.0,2.0,3.0,0.0,10.08,49,69265
300149,5.0,1481.0,1.0,0.0,0.0,5.0,3.0,0.0,10.42,49,77105
300150,5.0,1486.0,1.0,1.0,0.0,5.0,3.0,0.0,13.83,49,79099
300151,5.0,1483.0,1.0,1.0,0.0,2.0,3.0,0.0,10.00,49,81585
